In [ ]:
import pandas as pd
import geopandas as gpd
import glob
import os
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial

data_v = 'VersionD'
# 设定日期范围
date_start = '20230801'
date_end   = '20250730'

def process_single_csv(csv_file, polygon_gdf, csv_path):
    """处理单个CSV文件的函数（用于并行处理）"""
    try:
        # 读取CSV文件
        df = pd.read_csv(csv_file)
        
        # 创建点的GeoDataFrame
        points_gdf = gpd.GeoDataFrame(
            df, 
            geometry=gpd.points_from_xy(df.lon, df.lat),
            crs="EPSG:4326"
        )
        
        # 空间连接
        joined_gdf = gpd.sjoin(points_gdf, polygon_gdf, how='inner', predicate='within')
        
        if not joined_gdf.empty:
            print(f"处理完成: {os.path.basename(csv_file)}, 匹配到 {len(joined_gdf)} 个点")
            return joined_gdf
        else:
            print(f"处理完成: {os.path.basename(csv_file)}, 没有匹配的点")
            return None
    except Exception as e:
        print(f"处理 {csv_file} 时出错: {str(e)}")
        return None

def main():
    polygon_file = '../1-buffer-match/600mRiverReach-straightBuffer-corrected-20station.shp'
    if data_v == 'VersionD':
        csv_path = '/shared5/RESEARCH_DATA/SWOT/VersionD/processed'
    elif data_v == 'VersionC':
        csv_path = '/shared5/RESEARCH_DATA/SWOT/VersionC/processed'
    
    # 读取多边形数据（只读一次）
    print("读取多边形数据...")
    polygon_df = gpd.read_file(polygon_file)
    polygon_df = polygon_df.rename(columns={'stnmpy': 'stationid'})
    polygon_gdf = polygon_df[['stationid', 'COMID', 'geometry']].copy()
    
    # 坐标系处理
    if polygon_gdf.crs is None:
        polygon_gdf.set_crs("EPSG:4326", inplace=True)
    elif str(polygon_gdf.crs) != "EPSG:4326":
        polygon_gdf = polygon_gdf.to_crs("EPSG:4326")
    
    # 创建空间索引以加速空间查询
    print("创建空间索引...")
    polygon_gdf.sindex
    
    # 获取所有CSV文件
    all_csv_files = glob.glob(os.path.join(csv_path, 'node*.csv'))

    csv_files = sorted([
        f for f in all_csv_files
        if date_start <= os.path.basename(f).split('_')[1].replace('.csv', '') <= date_end
    ])
    print(f"找到 {len(csv_files)} 个CSV文件")
    
    # 使用多进程并行处理
    all_results = []
    max_workers = min(os.cpu_count() - 1, len(csv_files))  # 保留一个核心给系统
    
    print(f"使用 {max_workers} 个进程并行处理...")
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # 提交所有任务
        future_to_csv = {
            executor.submit(process_single_csv, csv_file, polygon_gdf, csv_path): csv_file 
            for csv_file in csv_files
        }
        
        # 收集结果
        for future in as_completed(future_to_csv):
            result = future.result()
            if result is not None:
                all_results.append(result)
    
    # 合并所有结果并保存
    if all_results:
        print("合并所有结果...")
        final_result = pd.concat(all_results, ignore_index=True)

        final_result['date'] = pd.to_datetime('2000-01-01') + pd.to_timedelta(final_result['time'], unit='s')
        final_result['date'] = final_result['date'].dt.date
        
        # 删除geometry列以节省空间（如果不需要）
        if 'geometry' in final_result.columns:
            final_result = final_result.drop(columns=['geometry'])
        
        output_file = os.path.join('./', f"1.all_matched_points_{data_v}.csv")
        final_result.to_csv(output_file, index=False)
        print(f"所有结果已保存到: {output_file}, 总共匹配到 {len(final_result)} 个点")
    else:
        print("没有找到任何匹配的点")

if __name__ == "__main__":
    main()

读取多边形数据...
创建空间索引...
找到 377 个CSV文件
使用 79 个进程并行处理...
处理完成: node_20230813.csv, 没有匹配的点
处理完成: node_20230820.csv, 没有匹配的点
处理完成: node_20230814.csv, 没有匹配的点
处理完成: node_20230815.csv, 没有匹配的点
处理完成: node_20230819.csv, 没有匹配的点
处理完成: node_20230818.csv, 没有匹配的点
处理完成: node_20230817.csv, 没有匹配的点
处理完成: node_20230812.csv, 没有匹配的点
处理完成: node_20230902.csv, 没有匹配的点
处理完成: node_20230903.csv, 没有匹配的点
处理完成: node_20230927.csv, 没有匹配的点
处理完成: node_20230913.csv, 没有匹配的点
处理完成: node_20230906.csv, 没有匹配的点
处理完成: node_20230828.csv, 匹配到 3 个点
处理完成: node_20230908.csv, 匹配到 5 个点
处理完成: node_20230914.csv, 没有匹配的点
处理完成: node_20230907.csv, 匹配到 2 个点
处理完成: node_20230904.csv, 没有匹配的点
处理完成: node_20230827.csv, 没有匹配的点
处理完成: node_20230909.csv, 匹配到 8 个点处理完成: node_20230824.csv, 没有匹配的点

处理完成: node_20230821.csv, 没有匹配的点
处理完成: node_20230910.csv, 匹配到 7 个点
处理完成: node_20230905.csv, 没有匹配的点
处理完成: node_20230912.csv, 没有匹配的点
处理完成: node_20230829.csv, 没有匹配的点
处理完成: node_20230831.csv, 匹配到 3 个点
处理完成: node_20230901.csv, 没有匹配的点
处理完成: node_20230822.csv, 没有匹配的点
处理完成: no